In [108]:
print('hello')

hello


## Best practices
1. Preprocessing And Cleaning 
2. Train Test Split 
3. Bow,TFIDF,Word2vec
4. Train ML algorithms

In [109]:
import pandas as pd 
df=pd.read_csv('dataset/kindle_reviews.csv')


In [110]:
df.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200


In [111]:
df=df[['reviewText','overall']]

In [112]:
df.head()

,reviewText,overall
0,I enjoy vintage books and movies so I enjoyed ...,5
1,This book is a reissue of an old one; the auth...,4
2,This was a fairly interesting read. It had ol...,4
3,I'd never read any of the Amy Brewster mysteri...,5
4,"If you like period pieces - clothing, lingo, y...",4


In [113]:
df.shape

(982619, 2)

In [114]:
##check missing values
df.isnull().sum()

reviewText    22
overall        0
dtype: int64

In [115]:
df['overall'].unique()

array([5, 4, 3, 2, 1])

In [116]:
#check index for null values
null_indexes = df[df.isnull().any(axis=1)].index
print(null_indexes.tolist())


[32652, 143683, 163443, 193140, 198536, 255972, 308493, 312057, 312310, 370839, 407344, 407624, 509549, 513873, 548273, 587053, 679623, 719610, 719613, 719620, 725868, 748161]


In [117]:
print(df.loc[null_indexes.tolist()])

       reviewText  overall
32652         NaN        5
143683        NaN        5
163443        NaN        5
193140        NaN        3
198536        NaN        5
255972        NaN        5
308493        NaN        4
312057        NaN        5
312310        NaN        5
370839        NaN        4
407344        NaN        5
407624        NaN        5
509549        NaN        5
513873        NaN        5
548273        NaN        5
587053        NaN        4
679623        NaN        5
719610        NaN        5
719613        NaN        5
719620        NaN        5
725868        NaN        5
748161        NaN        5


In [118]:
# Replace NaN in reviewText based on overall rating
df.loc[df['reviewText'].isna() & (df['overall'] == 5), 'reviewText'] = 'super'
df.loc[df['reviewText'].isna() & (df['overall'] == 4), 'reviewText'] = 'good'
df.loc[df['reviewText'].isna() & (df['overall'] == 3), 'reviewText'] = 'average'


In [119]:
print(df.loc[null_indexes.tolist()])

       reviewText  overall
32652       super        5
143683      super        5
163443      super        5
193140    average        3
198536      super        5
255972      super        5
308493       good        4
312057      super        5
312310      super        5
370839       good        4
407344      super        5
407624      super        5
509549      super        5
513873      super        5
548273      super        5
587053       good        4
679623      super        5
719610      super        5
719613      super        5
719620      super        5
725868      super        5
748161      super        5


In [120]:
#now check
df.isnull().sum()

reviewText    0
overall       0
dtype: int64

In [121]:
df.head()

,reviewText,overall
0,I enjoy vintage books and movies so I enjoyed ...,5
1,This book is a reissue of an old one; the auth...,4
2,This was a fairly interesting read. It had ol...,4
3,I'd never read any of the Amy Brewster mysteri...,5
4,"If you like period pieces - clothing, lingo, y...",4


In [122]:
df['overall'].value_counts()

overall
5    575264
4    254013
3     96194
2     34130
1     23018
Name: count, dtype: int64

# PREPROCESSING AND CLEANING

In [123]:
#if overall rating is greater than 3 -- positive rating and if it less than 3 -- negative rating

In [124]:
## positive review is 1 and negative review is 0 
df['overall']=df['overall'].apply(lambda x:0 if x<3 else 1)
df.head()

,reviewText,overall
0,I enjoy vintage books and movies so I enjoyed ...,1
1,This book is a reissue of an old one; the auth...,1
2,This was a fairly interesting read. It had ol...,1
3,I'd never read any of the Amy Brewster mysteri...,1
4,"If you like period pieces - clothing, lingo, y...",1


In [125]:
df['overall'].value_counts()

overall
1    925471
0     57148
Name: count, dtype: int64

In [126]:
## 1. Lower all the cases
df['reviewText']=df['reviewText'].str.lower()

In [127]:
df.head()

,reviewText,overall
0,i enjoy vintage books and movies so i enjoyed ...,1
1,this book is a reissue of an old one; the auth...,1
2,this was a fairly interesting read. it had ol...,1
3,i'd never read any of the amy brewster mysteri...,1
4,"if you like period pieces - clothing, lingo, y...",1


In [128]:
#2
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
# Ensure stopwords are downloaded
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [129]:
# Remove special characters (keep spaces)
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]+', ' ', str(x)))


<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ACER\AppData\Local\Temp\ipykernel_17000\2447771344.py:2: SyntaxWarning: invalid escape sequence '\s'
  df['reviewText'] = df['reviewText'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]+', ' ', str(x)))


In [130]:
# Remove URLs
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', str(x)))

In [131]:
# Remove HTML tags
df['reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(str(x), 'html.parser').get_text())

In [132]:
# Remove extra spaces (convert multiple spaces to single space)
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())


In [133]:
# Remove stopwords
stop_words = set(stopwords.words('english'))
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join([y for y in x.split() if y.lower() not in stop_words]))

In [134]:
df.head()

,reviewText,overall
0,enjoy vintage books movies enjoyed reading boo...,1
1,book reissue old one author born 1910 era say ...,1
2,fairly interesting read old style terminology ...,1
3,never read amy brewster mysteries one really h...,1
4,like period pieces clothing lingo enjoy myster...,1


In [136]:
## lemmatizer
from nltk.stem import WordNetLemmatizer

In [137]:
lemmatizer=WordNetLemmatizer()

In [138]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df['reviewText'] = df['reviewText'].apply(lambda x: lemmatize_words(x))


In [139]:
df.head()

,reviewText,overall
0,enjoy vintage book movie enjoyed reading book ...,1
1,book reissue old one author born 1910 era say ...,1
2,fairly interesting read old style terminology ...,1
3,never read amy brewster mystery one really hooked,1
4,like period piece clothing lingo enjoy mystery...,1


In [151]:
## Train Test split 
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(df['reviewText'],df['overall'],test_size=0.25)

In [160]:
##################### BOW
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer 
bow=CountVectorizer()
x_train_bow=bow.fit_transform(x_train)
x_test_bow=bow.transform(x_test)

In [161]:
##################### tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
x_train_tfidf=tfidf.fit_transform(x_train)
x_test_tfidf=tfidf.transform(x_test)

In [162]:
x_train_bow

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 32436112 stored elements and shape (736964, 228844)>

In [169]:
###now use ml model MultinomialNB
from sklearn.naive_bayes import MultinomialNB 
mnb_model_bow=MultinomialNB().fit(x_train_bow,y_train)
mnb_model_tfidf=MultinomialNB().fit(x_train_tfidf,y_train)

In [170]:
###performance
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [173]:
#prediction bow
y_pred_bow_test=mnb_model_bow.predict(x_test_bow)
y_pred_bow_train=mnb_model_bow.predict(x_train_bow)

In [174]:
#prediction tfidf
y_pred_tfidf_test=mnb_model_tfidf.predict(x_test_tfidf)
y_pred_tfidf_train=mnb_model_tfidf.predict(x_train_tfidf)

In [176]:
print('-------------------BOW------------------------------------------------')
print(f'Training accuracy:{accuracy_score(y_train,y_pred_bow_train)}')
print(f'Testing accuracy:{accuracy_score(y_test,y_pred_bow_test)}')
print(f'confusion matrix:{confusion_matrix(y_test,y_pred_bow_test)}')
print(f'classification report:{classification_report(y_test,y_pred_bow_test)}')

print('-------------------tfidf----------------------------------------------')
print(f'Training accuracy:{accuracy_score(y_train,y_pred_tfidf_train)}')
print(f'Testing accuracy:{accuracy_score(y_test,y_pred_tfidf_test)}')
print(f'confusion matrix:{confusion_matrix(y_test,y_pred_tfidf_test)}')
print(f'classification report :{classification_report(y_test,y_pred_tfidf_test)}')


-------------------BOW------------------------------------------------
Training accuracy:0.9417556352820491
Testing accuracy:0.9411206773727382
confusion matrix:[[  8470   5925]
 [  8539 222721]]
classification report:              precision    recall  f1-score   support

           0       0.50      0.59      0.54     14395
           1       0.97      0.96      0.97    231260

    accuracy                           0.94    245655
   macro avg       0.74      0.78      0.75    245655
weighted avg       0.95      0.94      0.94    245655

-------------------tfidf----------------------------------------------
Training accuracy:0.9417651337107376
Testing accuracy:0.9412631536097372
confusion matrix:[[    40  14355]
 [    74 231186]]
classification report :              precision    recall  f1-score   support

           0       0.35      0.00      0.01     14395
           1       0.94      1.00      0.97    231260

    accuracy                           0.94    245655
   macro avg      